<div class="alert alert-success" role="alert">
 <h1>Tarea 1</h1>
 <h2>Data Product Architecture</h2>
 <h2>Maestría en Ciencias de Datos, ITAM </h2>
 <h3>Ana Isabel Millán Careaga</h3>
 <h3>Esthefany Ortiz Gutiérrez</h3>
 <h3>Lorena Mariana Malpica Serrano</h3>
 <h3>Ollin Demian Langle Chimal</h3>
 <p>Fecha de entrega: Martes 7 de Febrero de 2017.</p>
 </div>

Mediante el siguiente código se bajan en paralelo los archivos comprimidos desde el 1ro de Diciembre del 2016 hasta la fecha del **gdeltproject** para después crear con ellos una base de datos con dos tablas, una que contiene toda la información de noticias donde los actores son de México y otra con 
+ Primero creamos una lista de las fechas en el formato en el que se encuentran en la página.
+ Vamos a descargar todos los archivos correspondientes a dichas fechas y los guardaremos en una carpeta llamada *archivos*.
+ Si no existe la carpeta baja los archivos, si existe no lo hace.
+ Tenemos el archivo de los nombres de las columnas y lo movemos a esa carpeta con un nombre de fecha de ceros para que se lea primero.
+ Leemos los archivos sin descomprimirlos y con *awk* hacemos que nos devuelva sólo los renglones donde las columnas 8 y 18 correspondan a MEX (porque es el código de país de los actores) y que en la columna 2 diga *SQLDATE*, esto último es sólo para que pueda leer el archivo de los nombres de las columnas, pudimos usar cualquier otra.
+ Usando el comando *tee* guardamos la salida a un archivo mientras al mismo tiempo la procesamos para ingresarla en una base de datos con una tabla
+ Aprovechamos el archivo creado y usando *awk* analizamos el número de eventos por día y su escala Goldstein promedio, la salida de esto lo guardamos en otra tabla dentro de la misma base de datos.
+ Borramos los archivos auxiliares

<div class="alert alert-info" role="alert">
<p class ="lead"> 
<h1>El programa es el siguiente </h1>
</div>

In [1]:
;cat ejecutable.sh

#!/usr/bin/env bash

d=; n=0; until [ "$d" = "20161201" ]; do ((n++)); d=$(date -d "today - $n days" +%Y%m%d); echo $d.export.CSV.zip >> fechas.txt;done

[ -d ./archivos ] || cat fechas.txt | parallel -j0 wget http://data.gdeltproject.org/events/\{\} -P ./archivos

zip 00000000.export.CSV.zip headers.csv && mv 00000000.export.CSV.zip ./archivos

parallel gunzip -c ::: $(ls ./archivos/*.export.CSV.zip) | awk '($2=="SQLDATE" || $8=="MEX" || $18=="MEX")  {print}' | tee -a salida.txt | csvsql --db sqlite:///gdelt.db --table mexico --insert -t 

awk -F "\t" '($57!="DATEADDED"){a[$57]+=$31;b[$57]+=1}END{for(i in a) print i "\t" b[i] "\t" a[i]/b[i]}' salida.txt >> gs.txt && sed -i "1iDate \t NumberOfOcurrences \t GoldsteinScale" gs.txt && cat gs.txt |  csvsql --db sqlite:///gdelt.db --table mexico_ts --insert -t

rm fechas.txt salida.txt gs.txt


<div class="alert alert-info" role="alert">
<p class ="lead"> 
<h1>Revisamos el peso de los archivos comprimidos. </h1>
</div>

In [2]:
;du -h archivos

815M	archivos


<div class="alert alert-info" role="alert">
<p class ="lead"> 
<h1>Revisamos el número de archivos.</h1>
</div>

In [3]:
;ls archivos/2*.zip | wc -l

67


31 de Diciembre + 31 de Enero + 5 de Febrero

<div class="alert alert-success" role="alert">
<h2>Podemos observar las tablas resultantes</h2>
</div>

In [4]:
using(SQLite)

In [5]:
db = SQLite.DB("./gdelt.db")

SQLite.DB("./gdelt.db")

In [6]:
SQLite.tables(db)

,name
1,"Nullable{String}(""mexico"")"
2,"Nullable{String}(""mexico_ts"")"


In [7]:
SQLite.query(db,"SELECT * FROM mexico";stricttypes=false)

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
1,Nullable{Any}(605043646),Nullable{Any}(20161204),Nullable{Any}(201612),Nullable{Any}(2016),Nullable{Any}(2016.92),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),"Nullable{Any}(""MEX"")","Nullable{Any}(""MEXICO"")","Nullable{Any}(""MEX"")",Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(1),Nullable{Any}(13),Nullable{Any}(13),Nullable{Any}(1),Nullable{Any}(1),Nullable{Any}(0.4),Nullable{Any}(14),Nullable{Any}(2),Nullable{Any}(14),Nullable{Any}(-1.30501),Nullable{Any}(0),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(1),"Nullable{Any}(""Mexico"")","Nullable{Any}(""MX"")","Nullable{Any}(""MX"")",Nullable{Any}(23),Nullable{Any}(-102),"Nullable{Any}(""MX"")",Nullable{Any}(1),"Nullable{Any}(""Mexico"")","Nullable{Any}(""MX"")","Nullable{Any}(""MX"")",Nullable{Any}(23),Nullable{Any}(-102),"Nullable{Any}(""MX"")",Nullable{Any}(20161204),"Nullable{Any}(""http://www.detroitnews.com/story/business/autos/ford/2016/12/03/ford-willing-work-trump-policies-right/94911186/"")"
2,Nullable{Any}(605045192),Nullable{Any}(20161204),Nullable{Any}(201612),Nullable{Any}(2016),Nullable{Any}(2016.92),"Nullable{Any}(""MEX"")","Nullable{Any}(""MEXICO"")","Nullable{Any}(""MEX"")",Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(1),Nullable{Any}(50),Nullable{Any}(50),Nullable{Any}(5),Nullable{Any}(1),Nullable{Any}(3.5),Nullable{Any}(10),Nullable{Any}(1),Nullable{Any}(10),Nullable{Any}(0.376648),Nullable{Any}(2),"Nullable{Any}(""Colorado, United States"")","Nullable{Any}(""US"")","Nullable{Any}(""USCO"")",Nullable{Any}(39.0646),Nullable{Any}(-105.327),"Nullable{Any}(""CO"")",Nullable{Any}(0),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(2),"Nullable{Any}(""Colorado, United States"")","Nullable{Any}(""US"")","Nullable{Any}(""USCO"")",Nullable{Any}(39.0646),Nullable{Any}(-105.327),"Nullable{Any}(""CO"")",Nullable{Any}(20161204),"Nullable{Any}(""http://www.havasunews.com/opinion/our-view-trump-rhetoric-no-help-to-arizona-s-trade/article_fdfe1b44-b9ac-11e6-b82a-874650666821.html"")"
3,Nullable{Any}(605045193),Nullable{Any}(20161204),Nullable{Any}(201612),Nullable{Any}(2016),Nullable{Any}(2016.92),"Nullable{Any}(""MEX"")","Nullable{Any}(""MEXICO"")","Nullable{Any}(""MEX"")",Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),"Nullable{Any}(""BRA"")","Nullable{Any}(""BRAZIL"")","Nullable{Any}(""BRA"")",Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(),Nullable{Any}(0),Nullable{Any}(61),Nullable{Any}(61),Nullable{Any}(6),Nul

In [8]:
SQLite.query(db,"SELECT * FROM mexico_ts")

,Date,NumberOfOcurrences,GoldsteinScale
1,Nullable{Int64}(20161201),Nullable{Int64}(1182),Nullable{Float64}(1.08985)
2,Nullable{Int64}(20161202),Nullable{Int64}(1055),Nullable{Float64}(1.19156)
3,Nullable{Int64}(20161203),Nullable{Int64}(547),Nullable{Float64}(0.160329)
4,Nullable{Int64}(20161204),Nullable{Int64}(423),Nullable{Float64}(0.483215)
5,Nullable{Int64}(20161205),Nullable{Int64}(803),Nullable{Float64}(0.99477)
6,Nullable{Int64}(20161206),Nullable{Int64}(886),Nullable{Float64}(1.24819)
7,Nullable{Int64}(20161207),Nullable{Int64}(809),Nullable{Float64}(0.869592)
8,Nullable{Int64}(20161208),Nullable{Int64}(1042),Nullable{Float64}(1.04981)
9,Nullable{Int64}(20161209),Nullable{Int64}(1100),Nullable{Float64}(0.773182)
10,Nullable{Int64}(20161210),Nullable{Int64}(606),Nullable{Float64}(0.797855)
